In [54]:
import pandas as pd

In [55]:
df = pd.read_excel('data.xlsx')

In [56]:
df

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90
...,...,...,...,...,...
9563,16.65,49.69,1014.01,91.00,460.03
9564,13.19,39.18,1023.67,66.78,469.62
9565,31.32,74.33,1012.92,36.48,429.57
9566,24.48,69.45,1013.86,62.39,435.74


In [57]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9568 entries, 0 to 9567
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      9568 non-null   float64
 1   V       9568 non-null   float64
 2   AP      9568 non-null   float64
 3   RH      9568 non-null   float64
 4   PE      9568 non-null   float64
dtypes: float64(5)
memory usage: 373.9 KB


In [58]:
df.isnull().sum()

AT    0
V     0
AP    0
RH    0
PE    0
dtype: int64

In [59]:
x = df[['AT','V','AP','RH']]
y = df['PE']

In [60]:
from sklearn.preprocessing import MinMaxScaler

In [61]:
scaler = MinMaxScaler()

In [62]:
X_scaled = scaler.fit_transform(x)


In [63]:
X_scaled

array([[0.37252125, 0.29181495, 0.77159119, 0.63820375],
       [0.66203966, 0.66903915, 0.6718634 , 0.44932976],
       [0.09348442, 0.24982206, 0.47686216, 0.8924933 ],
       ...,
       [0.83597734, 0.87135231, 0.49566939, 0.1463807 ],
       [0.64220963, 0.78451957, 0.51893096, 0.49369973],
       [0.56062323, 0.66120996, 0.60232616, 0.56715818]], shape=(9568, 4))

In [64]:
from sklearn.model_selection import train_test_split

In [65]:
x_train,x_test,y_train,y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=42)

In [66]:
import torch
import torch.nn as nn

In [67]:
class Weather(nn.Module):
    def __init__(self,input_dim):
        super(Weather,self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim,32),
            nn.ReLU(),
            nn.Linear(32,16),
            nn.ReLU(),
            nn.Linear(16,8),
            nn.ReLU(),
            nn.Linear(8,1),
        )
    def forward(self,x):
        return self.net(x)
    
input_dim = x_train.shape[1]
model = Weather(input_dim)

print(model)


Weather(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=16, bias=True)
    (3): ReLU()
    (4): Linear(in_features=16, out_features=8, bias=True)
    (5): ReLU()
    (6): Linear(in_features=8, out_features=1, bias=True)
  )
)


In [68]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# 5. Convert Data to PyTorch Tensors
# x_train is a NumPy array, y_train is a pandas Series, so use .values on y_train
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)

In [69]:
# 6. Define Loss Function and Optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [70]:
import matplotlib.pyplot as plt

# ... (all your previous code for setup) ...

# 7. Create the Training Loop
print("\nStarting Training...")
epochs = 10000
losses = [] # Create an empty list to store losses

for epoch in range(epochs):
    y_pred = model(x_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    
    # Store the loss value
    losses.append(loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

print("Training finished!")


Starting Training...
Epoch [100/10000], Loss: 204424.3125
Epoch [200/10000], Loss: 161497.5156
Epoch [300/10000], Loss: 30950.6211
Epoch [400/10000], Loss: 4308.6777
Epoch [500/10000], Loss: 3992.5537
Epoch [600/10000], Loss: 3677.8040
Epoch [700/10000], Loss: 3372.6038
Epoch [800/10000], Loss: 3083.1277
Epoch [900/10000], Loss: 2813.6279
Epoch [1000/10000], Loss: 2566.7837
Epoch [1100/10000], Loss: 2343.9771
Epoch [1200/10000], Loss: 2145.4993
Epoch [1300/10000], Loss: 1970.7303
Epoch [1400/10000], Loss: 1818.3057
Epoch [1500/10000], Loss: 1686.2717
Epoch [1600/10000], Loss: 1572.2643
Epoch [1700/10000], Loss: 1473.6862
Epoch [1800/10000], Loss: 1387.8859
Epoch [1900/10000], Loss: 1312.3217
Epoch [2000/10000], Loss: 1244.6946
Epoch [2100/10000], Loss: 1183.0397
Epoch [2200/10000], Loss: 1125.7683
Epoch [2300/10000], Loss: 1071.6667
Epoch [2400/10000], Loss: 1019.8584
Epoch [2500/10000], Loss: 969.7488
Epoch [2600/10000], Loss: 920.9612
Epoch [2700/10000], Loss: 873.2755
Epoch [2800/1

In [71]:
# --- 8. Evaluate the Model on Test Data ---

# First, prepare the test data by converting it to PyTorch tensors
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Set the model to evaluation mode
model.eval()

# Use torch.no_grad() to stop PyTorch from calculating gradients
with torch.no_grad():
    # Make predictions on the test data
    y_pred_test = model(x_test_tensor)

    # Calculate the loss on the test data
    test_loss = criterion(y_pred_test, y_test_tensor)
    
    # For regression, Root Mean Squared Error (RMSE) is often more interpretable
    # It's the square root of the MSE loss
    rmse = torch.sqrt(test_loss)

print(f"\nEvaluation on Test Data:")
print(f"Test Loss (MSE): {test_loss.item():.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse.item():.4f}")


Evaluation on Test Data:
Test Loss (MSE): 20.2737
Root Mean Squared Error (RMSE): 4.5026


In [72]:
# Import the required functions from scikit-learn
from sklearn.metrics import r2_score, mean_squared_error
import numpy as np

# --- Prerequisite: Your model has been evaluated ---
# The following variables should already exist from the previous evaluation step:
# y_test: Your original test data (pandas Series)
# y_pred_test: Your model's predictions (PyTorch tensor)

# --- Convert predictions tensor to a NumPy array ---
# Scikit-learn metrics work with NumPy arrays, not PyTorch tensors.
y_pred_numpy = y_pred_test.numpy()

# --- Calculate the metrics ---

# 1. R-squared Score
r2 = r2_score(y_test, y_pred_numpy)

# 2. Mean Squared Error (using scikit-learn)
mse = mean_squared_error(y_test, y_pred_numpy)

# 3. Root Mean Squared Error (take the square root of MSE)
rmse_sklearn = np.sqrt(mse)

print(f"\nMetrics calculated with scikit-learn:")
print(f"R-squared (R2 Score): {r2:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse_sklearn:.4f}")


Metrics calculated with scikit-learn:
R-squared (R2 Score): 0.9301
Root Mean Squared Error (RMSE): 4.5026


In [73]:
# Save the model's state dictionary
torch.save(model.state_dict(), 'weather_model.pth')
print("\nModel saved successfully!")


Model saved successfully!


In [74]:
import numpy as np

# --- Load the saved model ---

# First, you need to create an instance of the model architecture again
# Make sure input_dim matches the one you trained with (which is 4)
loaded_model = Weather(input_dim=4) 

# Load the saved weights into the model instance
loaded_model.load_state_dict(torch.load('weather_model.pth'))

# Set the model to evaluation mode
loaded_model.eval()

# --- Make a prediction on new, unseen data ---

# Example new data point: [AT, V, AP, RH]
new_data = np.array([[20.0, 60.0, 1015.0, 80.0]]) 

# IMPORTANT: You must scale the new data using the SAME scaler you used for training
scaled_new_data = scaler.transform(new_data)

# Convert the scaled data to a PyTorch tensor
new_data_tensor = torch.tensor(scaled_new_data, dtype=torch.float32)

# Use the loaded model to make a prediction
with torch.no_grad():
    predicted_pe = loaded_model(new_data_tensor)

print(f"\nNew Data Input: {new_data[0]}")
print(f"Predicted Power Output (PE): {predicted_pe.item():.4f}")


New Data Input: [  20.   60. 1015.   80.]
Predicted Power Output (PE): 451.4090


c:\Users\faisal\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [75]:
import joblib

# Assuming 'scaler' is the MinMaxScaler object you already trained
joblib.dump(scaler, 'scaler.pkl') 
print("Scaler saved!")

Scaler saved!
